In [1]:
from PIL import Image
import os
import sys
import pandas as pd

main_dir = os.path.abspath(os.pardir)
sys.path.insert(0, main_dir)

In [2]:
raw_imgs_pth = os.path.join(main_dir, "data", "raw")
intrm_img_pth = os.path.join(main_dir, "data", "interim")
os.makedirs(intrm_img_pth, exist_ok=True)
raw_imgs_pth, intrm_img_pth = raw_imgs_pth+os.sep, intrm_img_pth+os.sep
images_lst = [f for f in os.listdir(raw_imgs_pth) if os.path.isfile(os.path.join(raw_imgs_pth, f))]
images_lst = [f for f in images_lst if f.endswith(".jpg")]

# Conv, ts to greyscale, resizes, and crops
size = (480, 640)
box_coord = (40, 120)

(left, upper, right, lower) = (box_coord[0], box_coord[1], size[0]-box_coord[0], size[1]-box_coord[1])

start_time = pd.to_datetime("now")
print("Start time:", start_time.strftime("%Y/%m/%d %H:%M:%S"))
for i, image_fname in enumerate(images_lst):
    print("Progress: "+str(int(round(((i+1)/((len(images_lst)+1))*100), 0)))+"%", end="\r")
    try:
        img = Image.open(raw_imgs_pth+image_fname).convert('LA')
    except Exception as error:
        print(image_fname, repr(error))
        continue
    img.thumbnail(size, Image.BICUBIC)
    img = img.crop((left, upper, right, lower))
    img.save(intrm_img_pth+image_fname.replace(".jpg", "")+".png")
print()

elapsed_time = pd.to_datetime("now") - start_time
print("This has taken "+str(round(elapsed_time.seconds / 60, 0))+" minutes.")

Start time: 2020/05/09 21:09:57
Progress: 100%
This has taken 55.43 minutes.
